In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from platypus import Problem, EpsNSGAII, Real, ProcessPoolEvaluator
from platypus import (Solution, EpsilonBoxArchive, GenerationalDistance, InvertedGenerationalDistance,
                      Hypervolume, EpsilonIndicator, Spacing)
import pickle
import csv
import logging
from itertools import chain
logging.basicConfig(level=logging.INFO)

In [ ]:
# change directory to model location
os.chdir("./MUSEH2OPS") 
from susquehanna_model import susquehanna_model

In [ ]:
# run model, skip if results are already in output folder

#set seed
seeds = [10] # 345, 456, 567]
# seeds = [1234, 2345] [3456, 4567, 5678]
# RBFlist = ["multiQuadric", "multiQuadric2"] #["exponential", "matern32"] # ["gaussian", "SE", "invmultiquadric", "invquadratic"]
# for RBF in RBFlist:
for modelseed in seeds:
    random.seed(modelseed)
    RBFType = "SE"
    numberOfInput = 4 # (time, storage of Conowingo)
    numberOfOutput = 4  # Atomic, Baltimore,Chester, Downstream:- hydropower, environmental
    numberOfRBF = 6  # numberOfInput + 2
    # N = 2 * n * m + K * n  # check 32 with 2 inputs, 72 with 4 inputs

    # Initialize model
    nobjs = 6
    nvars = int(numberOfRBF * 8 + 2)  # 8 = 2 centers + 2 radius + 4 weights
    n_years = 1
    susquehanna_river = susquehanna_model(108.5, 505.0, 5, n_years)  # l0, l0_MR, d0, years
    # l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5
    susquehanna_river.load_data(0)  # 0 = historic, 1 = stochastic
    susquehanna_river.set_log(False)

    susquehanna_river.setRBF(numberOfRBF, numberOfInput, numberOfOutput, RBFType)

    # Lower and Upper Bound for problem.types
    LB = [-1, 0, -1, 0, 0, 0, 0, 0] * numberOfRBF + [0, 0]
    UB = [1, 1, 1, 1, 1, 1, 1, 1] * numberOfRBF + [np.pi * 2, np.pi * 2]

    # np.pi*2 for phaseshift upperbounds (J. Quinn Como model) check borg optimization_serial flood C model
    EPS = [0.5, 0.05, 0.05, 0.05, 0.05, 0.001]

    # platypus MOEA, no contraints
    problem = Problem(nvars, nobjs)
    # problem.types[:] = Real(-1, 1)
    problem.types[:] = [Real(LB[i], UB[i]) for i in range(nvars)]
    problem.function = susquehanna_river.evaluate  # historical (deterministic) optimization
    # problem.function = functools.partial(susquehanna_river.evaluates, opt_met=1) #way to add arguments
    # problem.function = susquehanna_river.evaluateMC  # stochastic optimization
    # problem.directions[:] = Problem.MINIMIZE
    problem.directions[0] = Problem.MINIMIZE  # hydropower
    problem.directions[1] = Problem.MINIMIZE  # atomicpowerplant
    problem.directions[2] = Problem.MINIMIZE  # baltimore
    problem.directions[3] = Problem.MINIMIZE  # chester
    problem.directions[4] = Problem.MAXIMIZE  # environment
    problem.directions[5] = Problem.MINIMIZE  # recreation

    ##added
    with ProcessPoolEvaluator(4) as evaluator:
        algorithm = EpsNSGAII(problem, epsilons=EPS, evaluator=evaluator)
        algorithm.run(10000)

    # results
#     print("results:")
#     for solution in algorithm.result:
#         print(solution.objectives)
#     with open(f"./output/{RBFType}_{modelseed}.pickle", "wb") as f:
#         pickle.dump(algorithm.result, f)
#     header = ['hydropower', 'atomicpowerplant', 'baltimore', 'chester', 'environment', 'recreation']
#     with open(f'{RBFType}_{modelseed}_solution.csv', 'w', encoding='UTF8', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(header)
#         for solution in algorithm.result:
#             writer.writerow(solution.objectives)

In [ ]:
# load variables
varlist = []
variables = []
for filename in os.listdir('output'):
    if filename.endswith('variables.csv'):
        varlist.append(filename[:-4])
        df_temp = pd.read_csv(f"output/{filename}", header=None)
        variables.append(df_temp.values.tolist())
print(f"Loaded: {', '.join(varlist)}")
variables = list(chain.from_iterable(variables))

In [ ]:
# Run model with found solution variables

# def model(RBFType, variables):
RBFType = "se" # RBF
numberOfInput = 4 # (time, storage of Conowingo)
numberOfOutput = 4  # Atomic, Baltimore,Chester, Downstream:- hydropower, environmental
numberOfRBF = 6  # numberOfInput + 2

# Initialize model
nobjs = 6
nvars = int(numberOfRBF * 8 + 2)  # 8 = 2 centers + 2 radius + 4 weights
n_years = 1
susquehanna_river = susquehanna_model(108.5, 505.0, 5, n_years)  # l0, l0_MR, d0, years
# l0 = start level cono, l0_MR = start level muddy run, d0 = startday > friday = 5
susquehanna_river.load_data(0)  # 0 = historic, 1 = stochastic
susquehanna_river.set_log(True)
susquehanna_river.setRBF(numberOfRBF, numberOfInput, numberOfOutput, RBFType)

# for solution in algorithm.result:
#     susquehanna_river.evaluate(solution.variables)
for nvars in variables:
    susquehanna_river.evaluate(nvars)
    
level_CO, level_MR, ratom, rbalt, rches, renv = susquehanna_river.get_log()
#     return level_CO, level_MR, ratom, rbalt, rches, renv

In [ ]:
# load demand
for filename in os.listdir('data1999'):
    if filename.startswith('w'):
        globals()[f"{filename[:-4]}"] = np.loadtxt(f'data1999/{filename}')

# plot releases and basins
alpha = 0.1
lw = 0.5
color = "blue"
for release in ratom:
   plot = plt.plot(release, color, linewidth=lw, alpha=alpha)
   plot = plt.plot(wAtomic, "black") 
   plot = plt.xlabel('days')
   plot = plt.ylabel('atom_release')
   plot = plt.ylim((0, 43))
plt.show()

for release in rbalt:
   plot = plt.plot(release, color, linewidth=lw, alpha=alpha)
   plot = plt.plot(wBaltimore, "black")
   plot = plt.xlabel('days')
   plot = plt.ylabel('baltimore_release')
   plot = plt.ylim((0, 495))
plt.show()

for release in rches:
   plot = plt.plot(release, color, linewidth=lw, alpha=alpha)
   plot = plt.plot(wChester, "black")
   plot = plt.xlabel('days')
   plot = plt.ylabel('chester_release')
   plot = plt.ylim((0, 57))
plt.show()

for release in renv:
   plot = plt.plot(release, color, linewidth=lw, alpha=alpha)
   plot = plt.xlabel('days')
   plot = plt.ylabel('environment_release')
   plot = plt.ylim((0, 259000))
plt.show()

for year in level_CO:
   plot = plt.plot(year, color, linewidth=lw, alpha=alpha)
   plot = plt.xlabel('days')
   plot = plt.ylabel('CO_level')
   plot = plt.ylim((101, 113))
plt.show()

for year in level_MR:
   plot = plt.plot(year, color, linewidth=lw, alpha=alpha)
   plot = plt.xlabel('days')
   plot = plt.ylabel('MR_level')
   plot = plt.ylim((466, 524))
plt.show()

In [ ]:
# load objectives
objlist = []
objectives = []
for filename in os.listdir('output'):
    if filename.endswith('solution.csv'):
        objlist.append(filename[:-4])
        df_temp = pd.read_csv(f"output/{filename}", header=0)
        objectives.append(df_temp.values.tolist())
print(f"Loaded: {', '.join(objlist)}")
objectives = list(chain.from_iterable(objectives))

In [ ]:
# parcoords plot objectives
dfl = []
for i in objectives:
    dfl.append(tuple(i))
df = pd.DataFrame(dfl, columns=['Hydropower', 'AtomicPowerplant', 'Baltimore', 'Chester', 'Environment', 'Recreation'])
df = df * -1
colors = []
for i in range(len(df)):
    colors.append(i)
fig = px.parallel_coordinates(df, color="Hydropower")
fig.show()